## Database: CITY VIEW

This tool set up different VIEWS (union of different indicators)

#----------------------------------------#

## DATABASE connection:

In [2]:
import numpy as np
import pandas as pd
import xarray as x
from configparser import ConfigParser
import sqlalchemy as sa # conection to the database

from sqlalchemy import create_engine, text
from datetime import datetime, timedelta



print (" import data")
### SET conection to MS-sql server:
################################################## SET postgre-sql connection:

################################################## read database keys:
def config(filename, section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception(
            'Section {0} not found in the {1} file'.format(section, filename))

    return db

keys = config(filename='database.ini')

POSTGRESQL_SERVER_NAME=keys['host']
PORT=                  keys['port']
Database_name =        keys['database']
USER =                 keys['user']
PSW =                  keys['password']
##################################################

engine_postgresql = sa.create_engine('postgresql://'+USER+':'+PSW+ '@'+POSTGRESQL_SERVER_NAME+':'+str(PORT)+ '/' + Database_name)
#print (engine_postgresql)
print ("Engine started..")

 import data
Engine started..


In [ ]:
####update timezone -- to be updated!!!!!

UPDATE lut.l_city_urau2021 

SET time_zone =

CASE WHEN cntr_code in 
		('AT','BE','CH','CY','CZ','DE','DK','ES','FR','HR','HU','IT','LU','MT','NL','NO','PL','SE','SI','SK') THEN 'Central European Time (UTC+1)' 
	WHEN cntr_code in 
		('BG','EE','EL','FI','LT','LV','RO') THEN 'Eastern European Time (UTC+2)' 	
	WHEN cntr_code in 
		('IE','IS','PT','UK') THEN 'Greenwich Mean Time(UTC)' 	END

	
UPDATE lut.l_city_urau2021 	
SET time_zone_offset=
CASE WHEN cntr_code in 
		('AT','BE','CH','CY','CZ','DE','DK','ES','FR','HR','HU','IT','LU','MT','NL','NO','PL','SE','SI','SK') THEN  1 
	WHEN cntr_code in 
		('BG','EE','EL','FI','LT','LV','RO') THEN 2	
	WHEN cntr_code in 
		('IE','IS','PT','UK') THEN 0		END 


#### (1.1)  VIEW I for the year 2018

In [4]:
#### (1.1)  VIEW !


################ OUTPUT VIEW name:
schema = "cube"
name_of_view = "city_view_01_2018"


### calc_indicator EC1020I and Export to a new table inside the database:
query = ("""      

  CREATE OR REPLACE VIEW """+ schema +"."+ name_of_view + """
 AS

SELECT 
                0 as env_zone_code,
                cu_city_eurostat.index, 
                cu_city_eurostat.urau_code, 
                cu_city_eurostat.estat_code, 
                cu_city_eurostat.city_code, 
                cu_city_eurostat.city_code_version, 
                cu_city_eurostat.parameter, 
                cu_city_eurostat.lineage, 
                cu_city_eurostat.datasource, 
                cu_city_eurostat.year, 
                cu_city_eurostat.parameter_value, 

                city_working_lat_lon_list.lat,
                city_working_lat_lon_list.lon,
                city_working_lat_lon_list.iso2,
                l_city_urau2021.urau_name,
                l_city_urau2021.nuts3_2021 AS area_sqm

                FROM cube.cu_city_eurostat
                LEFT JOIN lut.city_working_lat_lon_list  ON cu_city_eurostat.city_code = city_working_lat_lon_list.city_code
                LEFT JOIN lut.l_city_urau2021            ON cu_city_eurostat.city_code = city_working_lat_lon_list.urau_code
                        
                WHERE cube.cu_city_eurostat.city_code ~~ 'LU0%'::text


                union


                SELECT
                        cu_city_env_zones.env_zone_code,
                        99 as index,
                        cu_city_env_zones.urau_code, 
                        '' as estat_code, 
                        cu_city_env_zones.city_code, 
                        cu_city_env_zones.city_code_version, 
                        cu_city_env_zones.parameter, 
                        cu_city_env_zones.lineage, 
                        cu_city_env_zones.datasource, 
                        '2018' as year,
                        cu_city_env_zones.parameter_value, 

                        city_working_lat_lon_list.lat,
                        city_working_lat_lon_list.lon,
                        city_working_lat_lon_list.iso2,
                        l_city_urau2021.urau_name,
                        l_city_urau2021.nuts3_2021 AS area_sqm


                        FROM cube.cu_city_env_zones
                        LEFT JOIN lut.city_working_lat_lon_list  ON cu_city_env_zones.city_code = city_working_lat_lon_list.city_code
                        LEFT JOIN lut.l_city_urau2021            ON cu_city_env_zones.city_code = city_working_lat_lon_list.urau_code
                        
                WHERE cube.cu_city_env_zones.city_code ~~ 'LU0%'::text;

  




         """)
#print (query)
connection = engine_postgresql.raw_connection()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

print ("VIEW 1 produced!! ")


VIEW 1 produced!! 


In [10]:
#### (1.2)  VIEW !


################ OUTPUT VIEW name:
schema = "cube"
name_of_view = "city_view_02_2018"


### calc_indicator EC1020I and Export to a new table inside the database:
query = ("""      

  CREATE OR REPLACE VIEW """+ schema +"."+ name_of_view + """
 AS


SELECT  
l_city_urau2021.city_code, 
l_city_urau2021.urau_code, 
l_city_urau2021.urau_catg, 
l_city_urau2021.cntr_code, 
l_city_urau2021.urau_name, 
l_city_urau2021.fua_code, 
l_city_urau2021.area_sqm, 
l_city_urau2021.nuts3_2021, 
l_city_urau2021.shape_area / 10000.00 as city_area_ha,  
l_city_urau2021._wgs84x, 
l_city_urau2021._wgs84y, 
l_city_urau2021._laeax, 
l_city_urau2021._laeay ,


-- DEM:
--c_city_dem.dem_min, 
---c_city_dem.dem_max, 
c_city_dem.dem_mean, 
---c_city_dem.dem_std,
----- ENV Zone:
--c_city_env_zones.env_zone_code,
lut.l_env_zones.ez_code,
--- IMP:
100/l_city_urau2021.shape_area / 10000 *  c_city_hrl_imperviousness_2018.parameter_value as imd_percent_2018,
--- TREE:
100/l_city_urau2021.shape_area / 10000 *  c_city_hrl_treecover_2018.parameter_value as treecover_percent_2018,





--- Urban Atlas 2018
area_ha_11100,
area_ha_11210,
area_ha_11220,
area_ha_11230,
area_ha_11240,
area_ha_11300,
area_ha_12100,
area_ha_12210,
area_ha_12220,
area_ha_12230,
area_ha_12400,
area_ha_13100,
area_ha_13300,
area_ha_13400,
area_ha_14100,
area_ha_14200,
area_ha_21000,
area_ha_22000,
area_ha_23000,
area_ha_31000,
area_ha_32000,
area_ha_50000,

--- 2m temp 2018
temp_mean."2018" as avg_2m_temp_kelvin_2018,

--Number of sommer days 2018 :
c_city_era5_summer_days."2018" as number_of_summer_days_2018,

--Number of tropical nights :
c_city_era5_tropical_nights."2018" as number_of_tropical_nights_2018,

--Number of utci heat nights :
utci_heat_nights_2018,

--water in and around city:
CASE WHEN coalesce(clc_coastal_lagoons_42, 0) + coalesce(clc_estuaries_43, 0)+coalesce(clc_sea_and_ocean_44, 0)  > 0 THEN 1 else 0 end  as city_located_at_coast,


'test' as test 






FROM lut.l_city_urau2021 
--DEM:
left join cube.c_city_dem                      on c_city_dem.city_code             = l_city_urau2021.city_code
--ENV ZONES:
left join cube.c_city_env_zones                on c_city_env_zones.city_code       = l_city_urau2021.city_code
--ENV ZONES lookup table:               
left join lut.l_env_zones                      on l_env_zones.ez_raster_code = c_city_env_zones.parameter_value
--imperviousness_2018 :
left join cube.c_city_hrl_imperviousness_2018  on c_city_hrl_imperviousness_2018.city_code = l_city_urau2021.city_code
--tree cover_2018 :
left join cube.c_city_hrl_treecover_2018       on c_city_hrl_treecover_2018.city_code = l_city_urau2021.city_code



--Urban atlas area per class: 2018 :
left join 
		(
		SELECT 
		city_code,
		max(CASE WHEN ua12_18_code =  '11100' THEN (parameter_value) END)  as area_ha_11100,
		max(CASE WHEN ua12_18_code =  '11210' THEN (parameter_value) END)  as area_ha_11210,
		max(CASE WHEN ua12_18_code =  '11220' THEN (parameter_value) END)  as area_ha_11220,
		max(CASE WHEN ua12_18_code =  '11230' THEN (parameter_value) END)  as area_ha_11230,
		max(CASE WHEN ua12_18_code =  '11240' THEN (parameter_value) END)  as area_ha_11240,
		max(CASE WHEN ua12_18_code =  '11300' THEN (parameter_value) END)  as area_ha_11300,
		max(CASE WHEN ua12_18_code =  '12100' THEN (parameter_value) END)  as area_ha_12100,
		max(CASE WHEN ua12_18_code =  '12210' THEN (parameter_value) END)  as area_ha_12210,
		max(CASE WHEN ua12_18_code =  '12220' THEN (parameter_value) END)  as area_ha_12220,
		max(CASE WHEN ua12_18_code =  '12230' THEN (parameter_value) END)  as area_ha_12230,
		max(CASE WHEN ua12_18_code =  '12400' THEN (parameter_value) END)  as area_ha_12400,
		max(CASE WHEN ua12_18_code =  '13100' THEN (parameter_value) END)  as area_ha_13100,
		max(CASE WHEN ua12_18_code =  '13300' THEN (parameter_value) END)  as area_ha_13300,
		max(CASE WHEN ua12_18_code =  '13400' THEN (parameter_value) END)  as area_ha_13400,
		max(CASE WHEN ua12_18_code =  '14100' THEN (parameter_value) END)  as area_ha_14100,
		max(CASE WHEN ua12_18_code =  '14200' THEN (parameter_value) END)  as area_ha_14200,
		max(CASE WHEN ua12_18_code =  '21000' THEN (parameter_value) END)  as area_ha_21000,
		max(CASE WHEN ua12_18_code =  '22000' THEN (parameter_value) END)  as area_ha_22000,
		max(CASE WHEN ua12_18_code =  '23000' THEN (parameter_value) END)  as area_ha_23000,
		max(CASE WHEN ua12_18_code =  '31000' THEN (parameter_value) END)  as area_ha_31000,
		max(CASE WHEN ua12_18_code =  '32000' THEN (parameter_value) END)  as area_ha_32000,
		max(CASE WHEN ua12_18_code =  '50000' THEN (parameter_value) END)  as area_ha_50000
		FROM cube.c_city_urban_atlas_2018

			group by city_code
		) ua18
		on ua18.city_code = l_city_urau2021.city_code


--2m mean temp 2018 :
left join (                   
			SELECT city_code, "2018"
				FROM cube.c_city_era5_2m_temperature
				 where parameter_id =  'city_era5_2m_temperature_yearly_mean'

	) 	temp_mean
	on left(temp_mean.city_code,6)        = l_city_urau2021.city_code


--Number of sommer days 2018 :
left join cube.c_city_era5_summer_days                      on left(c_city_era5_summer_days.city_code,6)        = l_city_urau2021.city_code


--Number of tropical nights :
left join cube.c_city_era5_tropical_nights                    on left(c_city_era5_tropical_nights.city_code,6)            = l_city_urau2021.city_code

--Number of heat nights (UTCI):
left join (                   
			SELECT city_code, SUM("2018") as "utci_heat_nights_2018"
				FROM cube.c_city_utci_indicator
				where  utci_class	
				in 
				( 'Very Strong Heat Stress'
				,'Strong Heat Stress'
				,'Extreme Heat Stress')
					group by  city_code

	) 	utci_heat_nights
	on left(utci_heat_nights.city_code,6)        = l_city_urau2021.city_code


	

--water in and around city:
left join cube.c_city_water_classes_clc_2018                      on left(c_city_water_classes_clc_2018.city_code,6)        = l_city_urau2021.city_code

--eurostat - urban autdit city:


---left join cube.c_city_eurostat                      on left(c_city_eurostat.city_code,6)        = l_city_urau2021.city_code

---where lut.l_city_urau2021.city_code  like  'LU0%'
	
	
	

  




         """)
#print (query)
connection = engine_postgresql.raw_connection()
cursor = connection.cursor()
cursor.execute(query)
connection.commit()

print ("VIEW 2 produced!! ")


VIEW 2 produced!! 
